In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Spark APP").config('spark.ui.port', '4050').getOrCreate()
spark

In [9]:
# concatenates all csv files into one with python
import os
import glob

path = 'C:/Users/vinic/Documents/spark/empresas'
all_files = glob.glob(os.path.join(path, "*.csv"))

# save all files to csv
df = spark.read.csv(all_files, sep=';', inferSchema=True)

In [4]:
df.count()

4585679

In [4]:
df.show(5)

+----+--------------------+----+---+-------+---+----+
| _c0|                 _c1| _c2|_c3|    _c4|_c5| _c6|
+----+--------------------+----+---+-------+---+----+
| 306|FRANCAMAR REFRIGE...|2240| 49|   0,00|  1|null|
|1355|BRASILEIRO & OLIV...|2062| 49|   0,00|  5|null|
|4820|REGISTRO DE IMOVE...|3034| 32|   0,00|  5|null|
|5347|ROSELY APARECIDA ...|2135| 50|   0,00|  5|null|
|6846|BADU E FILHOS TEC...|2062| 49|4000,00|  1|null|
+----+--------------------+----+---+-------+---+----+
only showing top 5 rows



In [10]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

In [11]:
for index, colName in enumerate(empresasColNames):
    df = df.withColumnRenamed(f'_c{index}', colName)

df.show(5)

+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial|natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|        306|         FRANCAMAR REFRIGE...|             2240|                         49|                     0,00|               1|                       null|
|       1355|         BRASILEIRO & OLIV...|             2062|                         49|                     0,00|               5|                       null|
|       4820|         REGISTRO DE IMOVE...|             3034|                         32|                     0,00|               5|                       null|
|       5347|         ROSELY APARE

In [8]:
df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [12]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [13]:
df = df.withColumn('capital_social_da_empresa', f.regexp_replace('capital_social_da_empresa', ',', '.'))
df = df.withColumn('capital_social_da_empresa', f.col('capital_social_da_empresa').cast(DoubleType()))
df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [11]:
df.show(5)

+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial|natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|        306|         FRANCAMAR REFRIGE...|             2240|                         49|                      0.0|               1|                       null|
|       1355|         BRASILEIRO & OLIV...|             2062|                         49|                      0.0|               5|                       null|
|       4820|         REGISTRO DE IMOVE...|             3034|                         32|                      0.0|               5|                       null|
|       5347|         ROSELY APARE

# Convertendo String para Tipo Date

In [16]:
data = [{'data_str': ['20220110', '20220109', '20220108', '20220107']}]
df_demo = spark.createDataFrame(data)
df_demo.printSchema()

root
 |-- data_str: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [17]:
df_demo = df_demo.withColumn(
    'data_str', f.to_date(df_demo.data_str.cast(StringType()), 'yyyyMMdd')
)
df_demo.printSchema()

root
 |-- data_str: date (nullable = true)



# Realizando consultas

In [4]:
import pandas as pd

data = pd.DataFrame(
    {
        'nome': ['GISELLE PAULA GUIMARAES CASTRO', 'ELAINE GARCIA DE OLIVEIRA', 'JOAO CARLOS ABNER DE LOURDES', 'MARTA ZELI FERREIRA'],
        'idade': [15, 22, 43, 51]
    }
)

data.to_csv('./data.csv', index=False)

df_demo = spark.read.csv('./data.csv', header=True, inferSchema=True)

In [7]:
df_demo.select(
    f.concat_ws(
        ', ',
        f.substring_index('nome', ' ', -1),
        f.substring_index('nome', ' ', 1)
    ).alias('ident'), 'idade'
).show()

+----------------+-----+
|           ident|idade|
+----------------+-----+
| CASTRO, GISELLE|   15|
|OLIVEIRA, ELAINE|   22|
|   LOURDES, JOAO|   43|
| FERREIRA, MARTA|   51|
+----------------+-----+



In [16]:
# Ordenando Dados

df.select(
    'razao_social_nome_empresarial',
    'cnpj_basico',
    'porte_da_empresa',
    'capital_social_da_empresa'
).orderBy(['porte_da_empresa', 'capital_social_da_empresa'], ascending=[False, False]).show()

+-----------------------------+-----------+----------------+-------------------------+
|razao_social_nome_empresarial|cnpj_basico|porte_da_empresa|capital_social_da_empresa|
+-----------------------------+-----------+----------------+-------------------------+
|         AAIP AGRO NEGOCIO...|   19097654|               5|                   5.0E10|
|         ROMOTEX INVESTIME...|   22237511|               5|          4.0913781956E10|
|         EQUINOR BRASIL EN...|    4028583|               5|               3.22586E10|
|         ITB HOLDING BRASI...|    4274016|               5|        1.745743540422E10|
|                  M A C SILVA|    1113037|               5|          1.6100127405E10|
|         GVT PARTICIPACOES...|   10242813|               5|          1.3498790549E10|
|              CIMENTO ACU S/A|   17852181|               5|             1.1445792E10|
|            TVGD HOLDING LTDA|   34298743|               5|                   1.0E10|
|            HOLDING XS 1 S.A.|   38122269|

In [19]:
# Realizando filtros

# utilizando where
df.where('capital_social_da_empresa > 10000').show()

+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial|natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|      58970|         TOTAL CAR VEICULO...|             2062|                         49|                  20000.0|               1|                       null|
|      70273|         VANDA DA SILVA LA...|             2135|                         50|                  15000.0|               1|                       null|
|      74218|            V G SILVA TECIDOS|             2135|                         50|                  16000.0|               1|                       null|
|     116791|          AGNALDO ROC

In [23]:
# utilizando filter

df.filter(df.razao_social_nome_empresarial.startswith('LANCHONETE')).select('razao_social_nome_empresarial').show()

+-----------------------------+
|razao_social_nome_empresarial|
+-----------------------------+
|         LANCHONETE RIO VE...|
|         LANCHONETE VISTA ...|
|         LANCHONETE E CHUR...|
|         LANCHONETE BASILE...|
|         LANCHONETE ASTRO ...|
|         LANCHONETE E REST...|
|         LANCHONETE E CHOP...|
|         LANCHONETE CACIQU...|
|         LANCHONETE MEREND...|
|         LANCHONETE PELAND...|
|         LANCHONETE E PIZZ...|
|         LANCHONETE COOPER...|
|         LANCHONETE E SORV...|
|         LANCHONETE MEIA E...|
|         LANCHONETE UNIAO ...|
|         LANCHONETE E BORR...|
|         LANCHONETE QUATRO...|
|         LANCHONETE REFOGA...|
|         LANCHONETE BOB BU...|
|         LANCHONETE BAGAGE...|
+-----------------------------+
only showing top 20 rows



In [25]:
df.select('razao_social_nome_empresarial').filter(f.upper(df.razao_social_nome_empresarial).like('%RESTAURANTE%')).show(truncate=False)

+-------------------------------------------------------+
|razao_social_nome_empresarial                          |
+-------------------------------------------------------+
|RESTAURANTE IMIGRANTE PORTUGUES LTDA.                  |
|MORAIS & CARVALHO RESTAURANTE E PIZZARIA LTDA          |
|BAR E RESTAURANTE PAGANOTTO LTDA                       |
|RODRIGUES & RODRIGUES RESTAURANTE LTDA                 |
|TEXAS RANCH BAR RESTAURANTE PRODUCOES ARTISTICAS E CULT|
|V V SANTOS RESTAURANTE BAR E ATIV DESPORTIVAS LTDA     |
|BAR E RESTAURANTE CASA DA QUINTA LTDA                  |
|DON MUGO RESTAURANTE LTDA                              |
|MARIA ROZA DOS SANTOS- BAR E RESTAURANTE               |
|GERACAO DE OURO - BAR E RESTAURANTE LTDA               |
|ELIANIA A. CUSTODIO RESTAURANTE                        |
|DOCE ESPIGA RESTAURANTE LTDA                           |
|R. A. D. ABRIL RESTAURANTE                             |
|BAR E RESTAURANTE IULIANCHESE LTDA.                    |
|R. DA S. ARAU

In [27]:
# Agregações

df.select(
    'cnpj_basico', 'porte_da_empresa', 'capital_social_da_empresa'
    ).groupBy(
        'porte_da_empresa'
    ).agg(
        f.avg('capital_social_da_empresa').alias('capital_social_medio'),
        f.count('cnpj_basico').alias('frequencia')
    ).orderBy(
        'porte_da_empresa', ascending=True
    ).show()

+----------------+--------------------+----------+
|porte_da_empresa|capital_social_medio|frequencia|
+----------------+--------------------+----------+
|            null|    8.35421888053467|      5985|
|               1|  339994.53313507047|   3129043|
|               3|  2601001.7677092687|    115151|
|               5|   708660.4208249793|   1335500|
+----------------+--------------------+----------+



In [28]:
df.select('capital_social_da_empresa').summary().show()

+-------+-------------------------+
|summary|capital_social_da_empresa|
+-------+-------------------------+
|  count|                  4585679|
|   mean|        503694.5478542674|
| stddev|     2.1118691490537727E8|
|    min|                      0.0|
|    25%|                      0.0|
|    50%|                   1000.0|
|    75%|                   7000.0|
|    max|         3.22014670262E11|
+-------+-------------------------+



In [42]:
# joins

pd.DataFrame(
    {
        'nomes': ['CARLOS', 'IVO', 'MÁRCIA', 'LEILA', 'LETÍCIA'],
        'idade': [15, 14, 16, 17, 14]
    }
).to_csv('df_pandas_1.csv', index=False)

pd.DataFrame(
    {
        'nomes': ['CARLOS', 'MÁRCIA', 'LEILA', 'BRENO', 'LETÍCIA'],
        'notas': [10, 1, 10, 7, 9]
    }  
).to_csv('df_pandas_2.csv', index=False)

idade = spark.read.csv('./df_pandas_1.csv', inferSchema=True, header=True)
notas = spark.read.csv('./df_pandas_2.csv', inferSchema=True, header=True)

In [43]:
# inner join

idade.join(notas, how='inner').show()

+------+-----+-------+-----+
| nomes|idade|  nomes|notas|
+------+-----+-------+-----+
|CARLOS|   15| CARLOS|   10|
|CARLOS|   15| MÁRCIA|    1|
|CARLOS|   15|  LEILA|   10|
|CARLOS|   15|  BRENO|    7|
|CARLOS|   15|LETÍCIA|    9|
|   IVO|   14| CARLOS|   10|
|   IVO|   14| MÁRCIA|    1|
|   IVO|   14|  LEILA|   10|
|   IVO|   14|  BRENO|    7|
|   IVO|   14|LETÍCIA|    9|
|MÁRCIA|   16| CARLOS|   10|
|MÁRCIA|   16| MÁRCIA|    1|
|MÁRCIA|   16|  LEILA|   10|
|MÁRCIA|   16|  BRENO|    7|
|MÁRCIA|   16|LETÍCIA|    9|
| LEILA|   17| CARLOS|   10|
| LEILA|   17| MÁRCIA|    1|
| LEILA|   17|  LEILA|   10|
| LEILA|   17|  BRENO|    7|
| LEILA|   17|LETÍCIA|    9|
+------+-----+-------+-----+
only showing top 20 rows



In [44]:
# left join

idade.join(notas, how='left').show()

+------+-----+-------+-----+
| nomes|idade|  nomes|notas|
+------+-----+-------+-----+
|CARLOS|   15| CARLOS|   10|
|CARLOS|   15| MÁRCIA|    1|
|CARLOS|   15|  LEILA|   10|
|CARLOS|   15|  BRENO|    7|
|CARLOS|   15|LETÍCIA|    9|
|   IVO|   14| CARLOS|   10|
|   IVO|   14| MÁRCIA|    1|
|   IVO|   14|  LEILA|   10|
|   IVO|   14|  BRENO|    7|
|   IVO|   14|LETÍCIA|    9|
|MÁRCIA|   16| CARLOS|   10|
|MÁRCIA|   16| MÁRCIA|    1|
|MÁRCIA|   16|  LEILA|   10|
|MÁRCIA|   16|  BRENO|    7|
|MÁRCIA|   16|LETÍCIA|    9|
| LEILA|   17| CARLOS|   10|
| LEILA|   17| MÁRCIA|    1|
| LEILA|   17|  LEILA|   10|
| LEILA|   17|  BRENO|    7|
| LEILA|   17|LETÍCIA|    9|
+------+-----+-------+-----+
only showing top 20 rows



In [45]:
# right join

idade.join(notas, how='right').show()

+-------+-----+------+-----+
|  nomes|idade| nomes|notas|
+-------+-----+------+-----+
| CARLOS|   15|CARLOS|   10|
|    IVO|   14|CARLOS|   10|
| MÁRCIA|   16|CARLOS|   10|
|  LEILA|   17|CARLOS|   10|
|LETÍCIA|   14|CARLOS|   10|
| CARLOS|   15|MÁRCIA|    1|
|    IVO|   14|MÁRCIA|    1|
| MÁRCIA|   16|MÁRCIA|    1|
|  LEILA|   17|MÁRCIA|    1|
|LETÍCIA|   14|MÁRCIA|    1|
| CARLOS|   15| LEILA|   10|
|    IVO|   14| LEILA|   10|
| MÁRCIA|   16| LEILA|   10|
|  LEILA|   17| LEILA|   10|
|LETÍCIA|   14| LEILA|   10|
| CARLOS|   15| BRENO|    7|
|    IVO|   14| BRENO|    7|
| MÁRCIA|   16| BRENO|    7|
|  LEILA|   17| BRENO|    7|
|LETÍCIA|   14| BRENO|    7|
+-------+-----+------+-----+
only showing top 20 rows



In [46]:
# outer join

idade.join(notas, how='outer').show()

+------+-----+-------+-----+
| nomes|idade|  nomes|notas|
+------+-----+-------+-----+
|CARLOS|   15| CARLOS|   10|
|CARLOS|   15| MÁRCIA|    1|
|CARLOS|   15|  LEILA|   10|
|CARLOS|   15|  BRENO|    7|
|CARLOS|   15|LETÍCIA|    9|
|   IVO|   14| CARLOS|   10|
|   IVO|   14| MÁRCIA|    1|
|   IVO|   14|  LEILA|   10|
|   IVO|   14|  BRENO|    7|
|   IVO|   14|LETÍCIA|    9|
|MÁRCIA|   16| CARLOS|   10|
|MÁRCIA|   16| MÁRCIA|    1|
|MÁRCIA|   16|  LEILA|   10|
|MÁRCIA|   16|  BRENO|    7|
|MÁRCIA|   16|LETÍCIA|    9|
| LEILA|   17| CARLOS|   10|
| LEILA|   17| MÁRCIA|    1|
| LEILA|   17|  LEILA|   10|
| LEILA|   17|  BRENO|    7|
| LEILA|   17|LETÍCIA|    9|
+------+-----+-------+-----+
only showing top 20 rows



# PySpark SQL

In [47]:
df.createOrReplaceTempView('df_view')

In [55]:
spark.sql(
        """
            SELECT porte_da_empresa, MEAN(capital_social_da_empresa) AS media
            FROM df_view
            GROUP BY porte_da_empresa
            ORDER BY media DESC
        """
    ).show()

+----------------+------------------+
|porte_da_empresa|             media|
+----------------+------------------+
|               3|2601001.7677092687|
|               5| 708660.4208249793|
|               1|339994.53313507047|
|            null|  8.35421888053467|
+----------------+------------------+



In [ ]:
# write to csv

df.write.csv(
    path='./output',
    mode='overwrite',
    sep=',',
    header=True
)

# write to parquet

df.write.parquet(
    path='./output',
    mode='overwrite'
)

# write to orc

df.write.orc(
    path='./output',
    mode='overwrite'
)

In [ ]:
# repartition to create only one file
df.write.coalesce(1).csv(
    path='./output',
    mode='overwrite',
    sep=',',
    header=True
)

# parition by column
df.write.parquet(
    path='./output',
    mode='overwrite',
    partitionBy='porte_da_empresa'
)

In [62]:
spark.stop()